Based on:
* https://www.kaggle.com/dienhoa/ventillator-fastai-lb-0-168-no-kfolds-no-blend
* https://www.kaggle.com/lucasmorin/spectral-analysis-feature-engineering  
* https://www.kaggle.com/junkoda/pytorch-lstm-with-tensorflow-like-initialization/notebook

So, please upvote them first!

Changes from original
* Add more callbacks [EarlyStopping and SaveModelCallback]
* More features
* Add oof 
* Use folds 
* [L1Loss is MAE](https://pytorch.org/docs/stable/generated/torch.nn.L1Loss.html)

# Install FASTAI

In [ ]:
!pip install -Uqq fastai

# Import

In [ ]:
import gc
import os
import random
import numpy as np 
import pandas as pd 

# Torch
from torch.utils.data import Dataset
import torch
from torch.utils.data import DataLoader
import torch.nn as nn

# FASTAI
from fastai.data.core import DataLoaders
from fastai.learner import Learner
from fastai.callback.progress import ProgressCallback
from fastai.optimizer import OptimWrapper
from torch import optim
from fastai.losses import MSELossFlat, L1LossFlat,CrossEntropyLossFlat
from fastai.callback.schedule import Learner
# https://docs.fast.ai/callback.tracker.html
from fastai.callback.tracker import EarlyStoppingCallback, ReduceLROnPlateau, SaveModelCallback
from fastai.data.transforms import IndexSplitter

#scipy for FFT
import scipy.signal as signal
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq
from scipy.signal import hilbert, chirp
from scipy.signal import blackman

#sklearn
from sklearn.preprocessing import RobustScaler, normalize
from sklearn.model_selection import KFold

In [ ]:
np.random.seed(10000)
RANDOM_STATE = 10000

# Load source datasets

In [ ]:
df = pd.read_csv('/kaggle/input/ventilator-pressure-prediction/train.csv')
df_test = pd.read_csv('../input/ventilator-pressure-prediction/test.csv')

# UTILS

In [ ]:
%%time
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

# Feature Engineering

In [ ]:


dict_types = {
'id': np.int32,
'breath_id': np.int32,
'R': np.int8,
'C': np.int8,
'time_step': np.float32,
'u_in': np.float32,
'u_out': np.int8, 
'pressure': np.float32,
} 

all_pressure = np.sort(df.pressure.unique())
PRESSURE_MIN = all_pressure[0]
PRESSURE_MAX = all_pressure[-1]
PRESSURE_STEP = (all_pressure[1] - all_pressure[0])

## FFT features

In [ ]:
%time
ffta = lambda x: np.abs(fft(np.append(x.values,x.values[0]))[:80])
ffta.__name__ = 'ffta'

fftw = lambda x: np.abs(fft(np.append(x.values,x.values[0])*w)[:80])
fftw.__name__ = 'fftw'



N = 80
w = blackman(N+1)

def fft_features(df):
    print("Step FFT-features")
    df['fft_u_in'] = df.groupby('breath_id')['u_in'].transform(ffta)
    df['fft_u_in_w'] = df.groupby('breath_id')['u_in'].transform(fftw)
    df['analytical'] = df.groupby('breath_id')['u_in'].transform(hilbert)
    df['envelope'] = np.abs(df['analytical'])
    df['phase'] = np.angle(df['analytical'])
    df['unwrapped_phase'] = df.groupby('breath_id')['phase'].transform(np.unwrap)
    df['phase_shift1'] = df.groupby('breath_id')['unwrapped_phase'].shift(1).astype(np.float32)
    df['IF'] = df['unwrapped_phase'] - df['phase_shift1'].astype(np.float32)
    df = df.fillna(0)
    print("Finish adding FFT-features")
    df = df.drop('analytical',axis=1)
    return df

# Main features

In [ ]:
def add_features(df: pd.DataFrame): 
    df['area'] = df['time_step'] * df['u_in']
    df['area'] = df.groupby('breath_id')['area'].cumsum()
    
    df['u_in_cumsum'] = (df['u_in']).groupby(df['breath_id']).cumsum()
    
    df['u_in_lag1'] = df.groupby('breath_id')['u_in'].shift(1)

    df['u_in_lag_back1'] = df.groupby('breath_id')['u_in'].shift(-1)

    df['u_in_lag2'] = df.groupby('breath_id')['u_in'].shift(2)

    df['u_in_lag_back2'] = df.groupby('breath_id')['u_in'].shift(-2)

    df['u_in_lag3'] = df.groupby('breath_id')['u_in'].shift(3)

    df['u_in_lag_back3'] = df.groupby('breath_id')['u_in'].shift(-3)

    df['u_in_lag4'] = df.groupby('breath_id')['u_in'].shift(4)

    df['u_in_lag_back4'] = df.groupby('breath_id')['u_in'].shift(-4)

    df = df.fillna(0)
    print("Step-1...Completed")


    gc.collect()
    df['breath_id__u_in__max'] = df.groupby(['breath_id'])['u_in'].transform('max')
    df['breath_id__u_in__mean'] = df.groupby(['breath_id'])['u_in'].transform('mean')
    df['breath_id__u_in__diffmax'] = df.groupby(['breath_id'])['u_in'].transform('max') - df['u_in']
    df['breath_id__u_in__diffmean'] = df.groupby(['breath_id'])['u_in'].transform('mean') - df['u_in']
    print("Step-2...Completed")
    gc.collect()
    df['u_in_diff1'] = df['u_in'] - df['u_in_lag1']
    
    df['u_in_diff2'] = df['u_in'] - df['u_in_lag2']
    
    df['u_in_diff3'] = df['u_in'] - df['u_in_lag3']
    
    df['u_in_diff4'] = df['u_in'] - df['u_in_lag4']
    
    print("Step-3...Completed")
    
    df['one'] = 1
    df['count'] = (df['one']).groupby(df['breath_id']).cumsum()
    df['u_in_cummean'] =df['u_in_cumsum'] /df['count']
    
    df['breath_id_lag']=df['breath_id'].shift(1).fillna(0)
    df['breath_id_lag2']=df['breath_id'].shift(2).fillna(0)
    df['breath_id_lagsame']=np.select([df['breath_id_lag']==df['breath_id']],[1],0)
    df['breath_id_lag2same']=np.select([df['breath_id_lag2']==df['breath_id']],[1],0)
    df['breath_id__u_in_lag'] = df['u_in'].shift(1).fillna(0)
    df['breath_id__u_in_lag'] = df['breath_id__u_in_lag'] * df['breath_id_lagsame']
    df['breath_id__u_in_lag2'] = df['u_in'].shift(2).fillna(0)
    df['breath_id__u_in_lag2'] = df['breath_id__u_in_lag2'] * df['breath_id_lag2same']
    print("Step-4...Completed")
    gc.collect()
    
    g = df.groupby('breath_id')['u_in']
    
    df['time_step_diff'] = df.groupby('breath_id')['time_step'].diff().fillna(0)
    df['ewm_u_in_mean'] = g.ewm(halflife=9).mean().reset_index(level=0,drop=True)
    df['ewm_u_in_std'] = g.ewm(halflife=10).std().reset_index(level=0,drop=True)
    df['ewm_u_in_corr'] = g.ewm(halflife=10).corr().reset_index(level=0,drop=True)

    # adding this make notebook allocate all memory
    #df[["15_in_sum","15_in_min","15_in_max","15_in_mean"]] = (g.rolling(window=15,min_periods=1).agg({"15_in_sum":"sum","15_in_min":"min","15_in_max":"max","15_in_mean":"mean"}).reset_index(level=0,drop=True))
    
    df['rolling_10_mean'] = g.rolling(window=10, min_periods=1).mean()\
                             .reset_index(level=0, drop=True)
    df['rolling_10_max'] = g.rolling(window=10, min_periods=1).max()\
                            .reset_index(level=0, drop=True)
    df['rolling_10_std'] = g.rolling(window=10, min_periods=1).std()\
                            .reset_index(level=0, drop=True)

    df['expand_mean'] = g.expanding(2).mean()\
                         .reset_index(level=0, drop=True)
    df['expand_max'] = g.expanding(2).max()\
                        .reset_index(level=0, drop=True)
    df['expand_std'] = g.expanding(2).std()\
                        .reset_index(level=0, drop=True)
    
    print("Step-5...Completed")
    gc.collect()
    
    df['R'] = df['R'].astype(str)
    df['C'] = df['C'].astype(str)
    df['R__C'] = df["R"].astype(str) + '__' + df["C"].astype(str)
    df = pd.get_dummies(df)
    print("Step-6...Completed")
    return df

In [ ]:
print("Train data...\n")
train = add_features(df)
train = fft_features(train)
print("Shape of train df", train.shape)
del df
gc.collect()

print("\nTest data...\n")
test = add_features(df_test)
test = fft_features(test)
gc.collect()

In [ ]:
targets = train[['pressure']].to_numpy().reshape(-1, 80)
train.drop(['pressure','id', 'breath_id','one','count','breath_id_lag','breath_id_lag2','breath_id_lagsame','breath_id_lag2same'], axis=1, inplace=True)
test = test.drop(['id', 'breath_id','one','count','breath_id_lag','breath_id_lag2','breath_id_lagsame','breath_id_lag2same'], axis=1)

In [ ]:
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)

In [ ]:
RS = RobustScaler()
train = RS.fit_transform(train)
test = RS.transform(test)

In [ ]:
train = train.reshape(-1, 80, train.shape[-1])
test = test.reshape(-1, 80, train.shape[-1])

In [ ]:
idx = list(range(len(train)))

In [ ]:
train.shape[-2:]

# Dataset

In [ ]:
class VentilatorDataset(Dataset):
    def __init__(self, data, target):
        self.data = torch.from_numpy(data).float()
        if target is not None:
            self.targets = torch.from_numpy(target).float()
                
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if hasattr(self, 'targets'): return self.data[idx], self.targets[idx]
        else: return self.data[idx]

# Model

In [ ]:
class RNNModel(nn.Module):
    def __init__(self, input_size=64):
        hidden = [400, 300, 200, 100]
        super().__init__()
        self.lstm1 = nn.LSTM(input_size, hidden[0],
                             batch_first=True, bidirectional=True)
        self.lstm2 = nn.LSTM(2 * hidden[0], hidden[1],
                             batch_first=True, bidirectional=True)
        self.lstm3 = nn.LSTM(2 * hidden[1], hidden[2],
                             batch_first=True, bidirectional=True)
        self.lstm4 = nn.LSTM(2 * hidden[2], hidden[3],
                             batch_first=True, bidirectional=True)
        self.fc1 = nn.Linear(2 * hidden[3], 50)
        self.selu = nn.SELU()
        self.fc2 = nn.Linear(50, 1)
        self._reinitialize()

    def _reinitialize(self):
        """
        Tensorflow/Keras-like initialization
        """
        for name, p in self.named_parameters():
            if 'lstm' in name:
                if 'weight_ih' in name:
                    nn.init.xavier_uniform_(p.data)
                elif 'weight_hh' in name:
                    nn.init.orthogonal_(p.data)
                elif 'bias_ih' in name:
                    p.data.fill_(0)
                    n = p.size(0)
                    p.data[(n // 4):(n // 2)].fill_(1)
                elif 'bias_hh' in name:
                    p.data.fill_(0)
            elif 'fc' in name:
                if 'weight' in name:
                    nn.init.xavier_uniform_(p.data)
                elif 'bias' in name:
                    p.data.fill_(0)

    def forward(self, x):
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        x, _ = self.lstm3(x)
        x, _ = self.lstm4(x)
        x = self.fc1(x)
        x = self.selu(x)
        x = self.fc2(x)

        return x

In [ ]:
batch_size = 1024
submission = pd.read_csv('../input/ventilator-pressure-prediction/sample_submission.csv')
test_dataset = VentilatorDataset(test, None)
test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle=False)

# Train

In [ ]:
kf = KFold(n_splits=5, shuffle=True)
preds_fold = []
oof_df = []
for fold, (train_index, valid_index) in enumerate(kf.split(idx)):
    
    preds = []
    model = RNNModel(input_size=train.shape[-1]).to('cuda')
    print("FOLD:", fold)
    print(train_index)
    print(valid_index)

    train_input, valid_input = train[train_index], train[valid_index]
    train_targets, valid_targets = targets[train_index], targets[valid_index]

    train_dataset = VentilatorDataset(train_input, train_targets)
    valid_dataset = VentilatorDataset(valid_input, valid_targets)
    
    train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=False)
    valid_loader = DataLoader(valid_dataset, batch_size = batch_size, shuffle=False)
    
    dls = DataLoaders(train_loader, valid_loader)
    learn = Learner(dls, model, loss_func=L1LossFlat())
    callbacks = [SaveModelCallback(every_epoch=True, monitor='valid_loss',min_delta=0.1, fname=f"fastai_fold{fold+1}.pth"),
                 ReduceLROnPlateau(monitor='valid_loss', min_delta=0.5, patience=5),
                EarlyStoppingCallback(monitor='valid_loss', min_delta=0.0005, patience=5)]
    learn.fit_one_cycle(120, lr_max=2e-3, cbs=callbacks)
    
    valid_oof_dataset = VentilatorDataset(valid_input, None)
    valid_oof_loader = DataLoader(valid_oof_dataset, batch_size = batch_size, shuffle=False)
    oof_pred = []
    oof_sample = []
    
    with torch.no_grad():
        for data in valid_oof_loader:
            pred = model(data.to('cuda')).squeeze(-1).flatten()
            oof_pred.extend(pred.detach().cpu().numpy())
        oof_df.append({
            'pred': oof_pred,
            'target': valid_targets
        })
        for data in test_loader:
            pred = model(data.to('cuda')).squeeze(-1).flatten()
            preds.extend(pred.detach().cpu().numpy())
    preds_fold.append(preds)

# OOF

In [ ]:
target = []
pred = []
for i in range(len(oof_df)):
    target.extend(oof_df[i]['target'].flatten())
    pred.extend(oof_df[i]['pred'])
print(len(target))    
print(len(pred))
from sklearn.metrics import mean_absolute_error
print(mean_absolute_error(target,pred))
total = pd.DataFrame(data={'pred': pred, 'target': target})
total.to_csv('oof_score.csv',index=False)

# Submission

In [ ]:
preds_fold = np.array(preds_fold)
df_test['pressure'] = np.median(preds_fold, axis=0)
df_test[['id', 'pressure']].to_csv('submission.csv', index=False)

# EOF